In [3]:
import os
from pydantic import BaseModel, Field
from typing import List, Optional
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Verify API key is set
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set OPENAI_API_KEY in your .env file")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
# CREATE A SIMPLE PYDANTIC AI AGENT THAT will use chatgpt api to answer prompts 
# get api key from .env file


    
class SimpleAIAgent:
    def __init__(self, model: str = "gpt-3.5-turbo", temperature: float = 0.7):
        self.model = model
        self.temperature = temperature
        self.messages = []
        
    def add_system_message(self, content: str):
        """Add a system message to guide the AI's behavior"""
        self.messages.append({"role": "system", "content": content})
        
    def add_user_message(self, content: str):
        """Add a user message to the conversation"""
        self.messages.append({"role": "user", "content": content})
        
    def get_response(self, user_input: str) -> str:
        """Get a response from the AI for the given user input"""
        self.add_user_message(user_input)
        
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=self.temperature
        )
        
        assistant_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_message})
        return assistant_message

In [6]:
# Create an instance of the agent
agent = SimpleAIAgent()

# Set up the system message (guides the AI's behavior)
agent.add_system_message("You are a helpful AI assistant. Be concise and friendly.")

# Test the agent
response = agent.get_response("Hello! Can you tell me a short joke?")
print("AI:", response)

AI: Sure! Why couldn't the bicycle find its way home? Because it lost its bearings!


In [8]:
def chat_with_agent():
    print("AI Assistant initialized. Type 'quit' to exit.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'quit':
            break
        response = agent.get_response(user_input)
        print(f"\nAI: {response}")

# Uncomment to start the chat
# chat_with_agent()

In [10]:
# app.py
import streamlit as st
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set page config
st.set_page_config(
    page_title="AI Assistant",
    page_icon="🤖",
    layout="wide"
)

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Initialize session state for chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Sidebar for settings
with st.sidebar:
    st.title("⚙️ Settings")
    model = st.selectbox(
        "Choose a model",
        ["gpt-3.5-turbo", "gpt-4", "gpt-4.1-nano" ],
        index=0
    )
    temperature = st.slider(
        "Temperature",
        min_value=0.0,
        max_value=2.0,
        value=0.7,
        step=0.1,
        help="Higher values make output more random, lower values more deterministic"
    )

# Main chat interface
st.title("🤖 AI Assistant")
st.caption("Powered by OpenAI's API and Streamlit")

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("What's on your mind?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Display assistant response
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        
        # Get response from OpenAI
        for response in client.chat.completions.create(
            model=model,
            messages=[{"role": m["role"], "content": m["content"]} for m in st.session_state.messages],
            temperature=temperature,
            stream=True
        ):
            if response.choices[0].delta.content is not None:
                full_response += response.choices[0].delta.content
                message_placeholder.markdown(full_response + "▌")
        
        message_placeholder.markdown(full_response)
    
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": full_response})

2025-07-03 11:56:49.527 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 11:56:49.536 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 11:56:49.536 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-07-03 11:56:49.537 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 11:56:49.537 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 11:56:49.537 WARNING streamlit.runtime.scriptrunner_utils.script_run_c